In [1]:
import os
import warnings
import re

import math
import sympy
import scipy
import cobra
import csv
import numpy as np
import pandas as pd
from cobra.io import read_sbml_model
from cobra.medium import minimal_medium
from cobra.flux_analysis import (single_gene_deletion, single_reaction_deletion, 
                                 double_gene_deletion, double_reaction_deletion)

In [2]:
org_list_file = open('/Users/ashnanarain/Computational_Biology/Gut_Microbiome/Models.csv')
org_names = [(i.split())[0] for i in org_list_file.readlines()]
org_names

['Acidaminococcus_fermentans_DSM_20731',
 'Akkermansia_muciniphila_ATCC_BAA_835',
 'Alistipes_finegoldii_DSM_17242',
 'Alistipes_putredinis_DSM_17216',
 'Alistipes_shahii_WAL_8301',
 'Bacteroides_caccae_ATCC_43185',
 'Bacteroides_coprocola_M16_DSM_17136',
 'Bacteroides_eggerthii_DSM_20697',
 'Bacteroides_fragilis_NCTC_9343',
 'Bacteroides_intestinalis_341_DSM_17393',
 'Bacteroides_stercoris_ATCC_43183',
 'Bacteroides_thetaiotaomicron_VPI_5482',
 'Bacteroides_uniformis_ATCC_8492',
 'Bifidobacterium_adolescentis_ATCC_15703',
 'Bifidobacterium_bifidum_PRL2010',
 'Bifidobacterium_longum_NCC2705',
 'Bilophila_wadsworthia_3_1_6',
 'Blautia_wexlerae_DSM_19850',
 'Citrobacter_amalonaticus_Y19',
 'Clostridium_clostridioforme_CM201',
 'Collinsella_tanakaei_YIT_12063',
 'Coprococcus_catus_GD_7',
 'Desulfovibrio_desulfuricans_subsp_desulfuricans_DSM_642',
 'Desulfovibrio_piger_ATCC_29098',
 'Dialister_invisus_DSM_15470',
 'Dorea_longicatena_DSM_13814',
 'Enterobacter_cloacae_EcWSU1',
 'Enterococcu

In [3]:
lethals_pd = pd.DataFrame(columns = org_names)

In [4]:
medium_western_file = open('/Users/ashnanarain/Computational_Biology/Gut_Microbiome/Diets_and_Media/Diet_Western.csv', 'r')
medium_western = {}
for i in medium_western_file.readlines():
    
    met = i.strip().split(',')
    met[0] = met[0].replace('_e', '(e)')
    medium_western[met[0]] = float(met[1]) * float(met[2])

In [5]:
medium_DMEM_file = open('/Users/ashnanarain/Computational_Biology/Gut_Microbiome/Diets_and_Media/Diet_DMEM.csv', 'r')
medium_DMEM = {}
for i in medium_DMEM_file.readlines():
    
    met = i.strip().split(',')
    met[0] = met[0].replace('_e', '(e)')
    medium_DMEM[met[0]] = float(met[1])

FileNotFoundError: [Errno 2] No such file or directory: '/Users/ashnanarain/Computational_Biology/Gut_Microbiome/Diets_and_Media/Diet_DMEM.csv'

In [13]:
growths = []
for org in org_names:
    
    file = '/Users/ashnanarain/Computational_Biology/Gut_Microbiome/Models/' + org + '.xml'
  
    model = read_sbml_model(file)
    
    '''
    medium_western_matched = {}
    for i in medium_western.keys():
        if i not in dir(model.exchanges):
            pass
        else:
            medium_western_matched[i] = medium_western[i]
            
    model.medium = medium_western_matched
    
    medium_DMEM_matched = {}
    for i in medium_DMEM.keys():
        if i not in dir(model.exchanges):
            pass
        else:
            medium_DMEM_matched[i] = medium_DMEM[i]
            
    model.medium = medium_western_matched
    max_growth = model.slim_optimize()
    print(org, max_growth)
    growths.append([org, max_growth])
    '''
    '''
    non_exc_rxns = []
    for i in model.reactions:
        if i not in model.exchanges:
            non_exc_rxns.append(i)
    '''
    
    model.medium = minimal_medium(model, 1)
    sol = model.optimize()
    max_growth = sol.objective_value

    single_reaction_deletion_results = single_reaction_deletion(model)
    single_reaction_deletion_results
    single_lethal_reactions = []

    for i in range(len(single_reaction_deletion_results)):
        if single_reaction_deletion_results.loc[i]['growth'] < (0.1 * max_growth):
            single_lethal_reactions.append(single_reaction_deletion_results.loc[i]['ids'])

    lethals_pd[org] = pd.Series(single_lethal_reactions)
    print(len(single_lethal_reactions))


/Users/ashnanarain/anaconda3/envs/cobraenv/lib/python3.11/site-packages/cobra/medium/minimal_medium.py:112: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  medium = pd.Series()


Read LP format model from file /var/folders/dn/p3xbs4r94jqf_5q7_q53zj_c0000gn/T/tmp28wqpxk9.lp
Reading time = 0.00 seconds
: 1163 rows, 2462 columns, 9666 nonzeros
Read LP format model from file /var/folders/dn/p3xbs4r94jqf_5q7_q53zj_c0000gn/T/tmp5j8s55rj.lp
Reading time = 0.00 seconds
: 1163 rows, 2462 columns, 9666 nonzeros
Read LP format model from file /var/folders/dn/p3xbs4r94jqf_5q7_q53zj_c0000gn/T/tmpadn4layj.lp
Reading time = 0.01 seconds
: 1163 rows, 2462 columns, 9666 nonzeros
Read LP format model from file /var/folders/dn/p3xbs4r94jqf_5q7_q53zj_c0000gn/T/tmp6ycazgad.lp
Reading time = 0.01 seconds
: 1163 rows, 2462 columns, 9666 nonzeros
Read LP format model from file /var/folders/dn/p3xbs4r94jqf_5q7_q53zj_c0000gn/T/tmpwip0ojq3.lp
Reading time = 0.01 seconds
: 1163 rows, 2462 columns, 9666 nonzeros
Read LP format model from file /var/folders/dn/p3xbs4r94jqf_5q7_q53zj_c0000gn/T/tmptu2yrp9k.lp
Reading time = 0.01 seconds
: 1163 rows, 2462 columns, 9666 nonzeros
Read LP format m

/Users/ashnanarain/anaconda3/envs/cobraenv/lib/python3.11/site-packages/cobra/medium/minimal_medium.py:112: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  medium = pd.Series()


Read LP format model from file /var/folders/dn/p3xbs4r94jqf_5q7_q53zj_c0000gn/T/tmpp6_zvk58.lp
Reading time = 0.00 seconds
: 1105 rows, 2430 columns, 9724 nonzeros
Read LP format model from file /var/folders/dn/p3xbs4r94jqf_5q7_q53zj_c0000gn/T/tmp6vtct_qt.lp
Reading time = 0.00 seconds
: 1105 rows, 2430 columns, 9724 nonzeros
Read LP format model from file /var/folders/dn/p3xbs4r94jqf_5q7_q53zj_c0000gn/T/tmp_j67i1db.lp
Reading time = 0.00 seconds
: 1105 rows, 2430 columns, 9724 nonzeros
Read LP format model from file /var/folders/dn/p3xbs4r94jqf_5q7_q53zj_c0000gn/T/tmp0kfbs_2h.lp
Reading time = 0.00 seconds
: 1105 rows, 2430 columns, 9724 nonzeros
Read LP format model from file /var/folders/dn/p3xbs4r94jqf_5q7_q53zj_c0000gn/T/tmpvs_qyksu.lp
Reading time = 0.00 seconds
: 1105 rows, 2430 columns, 9724 nonzeros
Read LP format model from file /var/folders/dn/p3xbs4r94jqf_5q7_q53zj_c0000gn/T/tmpjmc8ls48.lp
Reading time = 0.00 seconds
: 1105 rows, 2430 columns, 9724 nonzeros
Read LP format m

/Users/ashnanarain/anaconda3/envs/cobraenv/lib/python3.11/site-packages/cobra/medium/minimal_medium.py:112: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  medium = pd.Series()


...

Read LP format model from file /var/folders/dn/p3xbs4r94jqf_5q7_q53zj_c0000gn/T/tmp0_tahhue.lp
Reading time = 0.00 seconds
: 1006 rows, 2322 columns, 9908 nonzeros
Read LP format model from file /var/folders/dn/p3xbs4r94jqf_5q7_q53zj_c0000gn/T/tmpkezmtbsu.lp
Reading time = 0.00 seconds
: 1006 rows, 2322 columns, 9908 nonzeros
Read LP format model from file /var/folders/dn/p3xbs4r94jqf_5q7_q53zj_c0000gn/T/tmpqqj18rgf.lp
Reading time = 0.01 seconds
: 1006 rows, 2322 columns, 9908 nonzeros
Read LP format model from file /var/folders/dn/p3xbs4r94jqf_5q7_q53zj_c0000gn/T/tmpa6qwafoj.lp
Reading time = 0.00 seconds
: 1006 rows, 2322 columns, 9908 nonzeros
Read LP format model from file /var/folders/dn/p3xbs4r94jqf_5q7_q53zj_c0000gn/T/tmpe33lpse0.lp
Reading time = 0.00 seconds
: 1006 rows, 2322 columns, 9908 nonzeros
Read LP format model from file /var/folders/dn/p3xbs4r94jqf_5q7_q53zj_c0000gn/T/tmpt8xs6v6s.lp
Reading time = 0.01 seconds
: 1006 rows, 2322 columns, 9908 nonzeros
Read LP format m

/Users/ashnanarain/anaconda3/envs/cobraenv/lib/python3.11/site-packages/cobra/medium/minimal_medium.py:112: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  medium = pd.Series()


Read LP format model from file /var/folders/dn/p3xbs4r94jqf_5q7_q53zj_c0000gn/T/tmp12_3a8qr.lp
Reading time = 0.00 seconds
: 957 rows, 2076 columns, 8892 nonzeros
Read LP format model from file /var/folders/dn/p3xbs4r94jqf_5q7_q53zj_c0000gn/T/tmp8yd1olob.lp
Reading time = 0.00 seconds
: 957 rows, 2076 columns, 8892 nonzeros
Read LP format model from file /var/folders/dn/p3xbs4r94jqf_5q7_q53zj_c0000gn/T/tmpuvvyvetr.lp
Reading time = 0.00 seconds
: 957 rows, 2076 columns, 8892 nonzeros
Read LP format model from file /var/folders/dn/p3xbs4r94jqf_5q7_q53zj_c0000gn/T/tmp92_uxvn_.lp
Reading time = 0.00 seconds
: 957 rows, 2076 columns, 8892 nonzeros
Read LP format model from file /var/folders/dn/p3xbs4r94jqf_5q7_q53zj_c0000gn/T/tmp0x0pphet.lp
Reading time = 0.00 seconds
: 957 rows, 2076 columns, 8892 nonzeros
Read LP format model from file /var/folders/dn/p3xbs4r94jqf_5q7_q53zj_c0000gn/T/tmpewqeghmm.lp
Reading time = 0.00 seconds
: 957 rows, 2076 columns, 8892 nonzeros
Read LP format model f

/Users/ashnanarain/anaconda3/envs/cobraenv/lib/python3.11/site-packages/cobra/medium/minimal_medium.py:112: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  medium = pd.Series()


Read LP format model from file /var/folders/dn/p3xbs4r94jqf_5q7_q53zj_c0000gn/T/tmpzrdstq3f.lp
Reading time = 0.00 seconds
: 1169 rows, 2520 columns, 10118 nonzeros
Read LP format model from file /var/folders/dn/p3xbs4r94jqf_5q7_q53zj_c0000gn/T/tmp27h3axm6.lp
Reading time = 0.00 seconds
: 1169 rows, 2520 columns, 10118 nonzeros
Read LP format model from file /var/folders/dn/p3xbs4r94jqf_5q7_q53zj_c0000gn/T/tmpdx1wq8gx.lp
Reading time = 0.01 seconds
: 1169 rows, 2520 columns, 10118 nonzeros
Read LP format model from file /var/folders/dn/p3xbs4r94jqf_5q7_q53zj_c0000gn/T/tmpaxyz5qn5.lp
Reading time = 0.01 seconds
: 1169 rows, 2520 columns, 10118 nonzeros
Read LP format model from file /var/folders/dn/p3xbs4r94jqf_5q7_q53zj_c0000gn/T/tmp04w42f0o.lp
Reading time = 0.01 seconds
: 1169 rows, 2520 columns, 10118 nonzeros
Read LP format model from file /var/folders/dn/p3xbs4r94jqf_5q7_q53zj_c0000gn/T/tmpya2xmy8g.lp
Reading time = 0.01 seconds
: 1169 rows, 2520 columns, 10118 nonzeros
Read LP fo

In [12]:
sol.objective_value

1.0

In [14]:
file = open('FINAL_ALL_Single_Species_Lethals.csv', 'w')
lethals_pd.to_csv(file, index=False)

In [64]:
file = open('FINAL_Single_Species_Growths_DMEM.csv', 'w')
growths_df = pd.DataFrame(growths)
print(growths_df)
growths_df.to_csv(file, index=False)

                                          0         1
0                 Alistipes_shahii_WAL_8301  0.001178
1             Bacteroides_caccae_ATCC_43185  0.044971
2       Bacteroides_coprocola_M16_DSM_17136  0.014406
3            Bacteroides_fragilis_NCTC_9343  0.048726
4    Bacteroides_intestinalis_341_DSM_17393  0.022847
5     Bacteroides_thetaiotaomicron_VPI_5482  0.052550
6           Bacteroides_uniformis_ATCC_8492  0.037316
7   Bifidobacterium_adolescentis_ATCC_15703  0.032361
8            Bifidobacterium_longum_NCC2705  0.010528
9         Clostridium_clostridioforme_CM201  0.031560
10           Collinsella_tanakaei_YIT_12063  0.024553
11  Escherichia_coli_str_K_12_substr_MG1655  0.071480
12        Faecalibacterium_prausnitzii_L2_6  0.044971
13          Haemophilus_parainfluenzae_T3T1  0.025930
14           Megasphaera_elsdenii_DSM_20460  0.085024
15     Parabacteroides_distasonis_ATCC_8503  0.052550
16      Parabacteroides_johnsonii_DSM_18315  0.010944
17                 Prevotell